In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


A.3 Broadcasting trong NumPy

Bước 1 – Import thư viện và tạo dữ liệu

In [2]:
import numpy as np
from numpy.random import default_rng
rng = default_rng(seed=42)

# Tạo mảng 4x3 ngẫu nhiên
arr = rng.standard_normal((4, 3))
print("Mảng gốc:\n", np.round(arr, 4))


Mảng gốc:
 [[ 0.3047 -1.04    0.7505]
 [ 0.9406 -1.951  -1.3022]
 [ 0.1278 -0.3162 -0.0168]
 [-0.853   0.8794  0.7778]]


Bước 2 – Demean theo cột (Broadcast theo axis=0)

In [3]:
# Tính trung bình theo từng cột
col_means = arr.mean(0)
print("\nTrung bình mỗi cột:", np.round(col_means, 4))

# Trừ trung bình cột (broadcast 1D -> 2D)
demeaned_col = arr - col_means
print("\nMảng sau khi demean theo cột:\n", np.round(demeaned_col, 4))
print("\nTrung bình sau khi demean:", np.round(demeaned_col.mean(0), 4))



Trung bình mỗi cột: [ 0.13   -0.607   0.0523]

Mảng sau khi demean theo cột:
 [[ 0.1747 -0.433   0.6981]
 [ 0.8105 -1.3441 -1.3545]
 [-0.0022  0.2907 -0.0691]
 [-0.9831  1.4864  0.7255]]

Trung bình sau khi demean: [ 0. -0.  0.]


Bước 3 – Demean theo hàng (Broadcast theo axis=1)

In [7]:
# Tính trung bình theo hàng
row_means = arr.mean(1)
print("\nTrung bình mỗi hàng:", np.round(row_means, 4))

# Cần reshape về (4,1) để broadcast theo trục 1
row_means_reshaped = row_means.reshape((4,1))
demeaned_row = arr - row_means_reshaped
print("\nMảng sau khi demean theo hàng:\n", np.round(demeaned_row, 4))
print("\nTrung bình từng hàng sau demean:", np.round(demeaned_row.mean(1), 4))



Trung bình mỗi hàng: [ 0.0051 -0.7709 -0.0684  0.268 ]

Mảng sau khi demean theo hàng:
 [[ 0.2997 -1.045   0.7454]
 [ 1.7114 -1.1802 -0.5313]
 [ 0.1962 -0.2478  0.0516]
 [-1.1211  0.6113  0.5097]]

Trung bình từng hàng sau demean: [ 0. -0. -0.  0.]


Bước 4 – Broadcasting trong mảng 3D

In [4]:
# Mảng 3 chiều
arr3d = rng.standard_normal((3, 4, 5))
print("\nShape arr3d:", arr3d.shape)

# Trung bình theo chiều sâu (axis=2)
depth_means = arr3d.mean(2)
print("Shape depth_means:", depth_means.shape)

# Broadcasting depth_means (3x4) → arr3d (3x4x5)
demeaned3d = arr3d - depth_means[:, :, np.newaxis]

# Kiểm tra kết quả
print("\nMean theo chiều sâu sau demean:", np.round(demeaned3d.mean(2), 4))



Shape arr3d: (3, 4, 5)
Shape depth_means: (3, 4)

Mean theo chiều sâu sau demean: [[ 0.  0.  0. -0.]
 [ 0.  0. -0.  0.]
 [-0.  0.  0.  0.]]


Bước 5 – Hàm tổng quát hóa demean_axis()

In [6]:
def demean_axis(arr, axis=0):
    means = arr.mean(axis)
    indexer = [slice(None)] * arr.ndim
    indexer[axis] = np.newaxis  # thêm chiều broadcast
    return arr - means[tuple(indexer)]  # ✅ ép list -> tuple


# Test hàm
demo = np.arange(12).reshape(3,4)
print("\nMảng demo:\n", demo)
print("\nDemean theo trục 1:\n", demean_axis(demo, axis=1))



Mảng demo:
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]

Demean theo trục 1:
 [[-1.5 -0.5  0.5  1.5]
 [-1.5 -0.5  0.5  1.5]
 [-1.5 -0.5  0.5  1.5]]


A.4. Sử dụng nâng cao các hàm ufunc trong NumPy

cộng toàn bộ phần tử trong mảng bằng cách dùng np.add.reduce:

In [8]:
arr = np.arange(10)
np.add.reduce(arr)



np.int64(45)

Tương đương với:

In [9]:
arr.sum()


np.int64(45)

Kiểm tra từng hàng có được sắp xếp hay không

In [10]:
my_rng = np.random.default_rng(12346)
arr = my_rng.standard_normal((5, 5))
arr[::2].sort(1)  # sắp xếp một vài hàng
np.logical_and.reduce(arr[:, :-1] < arr[:, 1:], axis=1)
# Kết quả: array([True, False, True, False, True])


array([ True, False,  True, False,  True])

2. Phương thức accumulate

In [11]:
arr = np.arange(15).reshape((3,5))
np.add.accumulate(arr, axis=1)


array([[ 0,  1,  3,  6, 10],
       [ 5, 11, 18, 26, 35],
       [10, 21, 33, 46, 60]])

3. Phương thức outer

In [12]:
arr = np.arange(3).repeat([1, 2, 2])
np.multiply.outer(arr, np.arange(5))


array([[0, 0, 0, 0, 0],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 2, 4, 6, 8],
       [0, 2, 4, 6, 8]])

    Hoặc

In [13]:
x, y = rng.standard_normal((3,4)), rng.standard_normal(5)
result = np.subtract.outer(x, y)
result.shape  # (3,4,5)


(3, 4, 5)

4. Phương thức reduceat

In [14]:
arr = np.arange(10)
np.add.reduceat(arr, [0, 5, 8])



array([10, 18, 17])

Giải thích:

arr[0:5] → tổng = 10

arr[5:8] → tổng = 18

arr[8:] → tổng = 17

Có thể áp dụng theo trục cụ thể (axis):

In [18]:
arr = np.multiply.outer(np.arange(4), np.arange(5))
np.add.reduceat(arr, [0, 2, 4], axis=1)


array([[ 0,  0,  0],
       [ 1,  5,  4],
       [ 2, 10,  8],
       [ 3, 15, 12]])

A.4.2. Viết ufunc mới trong Python

Cách 1- Tạo ufunc bằng frompyfunc

In [15]:
def add_elements(x, y):
    return x + y

add_them = np.frompyfunc(add_elements, 2, 1)
add_them(np.arange(8), np.arange(8))


array([0, 2, 4, 6, 8, 10, 12, 14], dtype=object)

Cách 2 – Dùng np.vectorize (có thể chọn kiểu dữ liệu output)

In [16]:
add_them = np.vectorize(add_elements, otypes=[np.float64])
add_them(np.arange(8), np.arange(8))


array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14.])

So sánh hiệu năng

In [17]:
arr = rng.standard_normal(10000)
%timeit add_them(arr, arr)
# ~ 2.43 ms mỗi vòng

%timeit np.add(arr, arr)
# ~ 2.88 µs mỗi vòng


1.99 ms ± 91.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
9.69 µs ± 801 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
